In [1]:
import pandas as pd
import pickle

In [2]:
# Read in data

# Block to block travel population:

# Total number of jobs: S000
# Age columns: SA01, SA02, SA03 (all sum to S000)
# Income columns: SE01, SE02, SE03 (all sum to S000)
# Industry columns: SI01, SI02, SI03 (all sum to S000)

Origin_Destination = pd.read_csv('ca_od_main_JT00_2020.csv')

# Nodes with matching blocks:
with open('Node_Block.pkl', 'rb') as f:
    Node_Block = pickle.load(f)

In [3]:
print(Origin_Destination.head())

        w_geocode       h_geocode  S000  SA01  SA02  SA03  SE01  SE02  SE03  \
0  60014001001003  60014010006013     1     0     0     1     0     1     0   
1  60014001001003  60014039003005     1     0     0     1     0     0     1   
2  60014001001003  60014046003004     1     0     1     0     0     0     1   
3  60014001001003  60014206003002     1     0     0     1     0     0     1   
4  60014001001003  60014229021007     1     0     1     0     0     0     1   

   SI01  SI02  SI03  createdate  
0     0     0     1    20230321  
1     0     0     1    20230321  
2     0     0     1    20230321  
3     0     0     1    20230321  
4     0     0     1    20230321  


In [4]:
# Show unique values in the 'S000' column
print(Origin_Destination['S000'].unique())

[  1   2   3   4   6   5   7  11   8   9  12  10  13  17  56  46  15  14
  16  24  18  21  29  19  26  25  22  31  30  23  32  20  49  50  37  81
  33  27  34  44  39  36  35  71  38  48  53  28  51  45  96  47 117  75
  58  89  94  54  43  74  55  63  65  42  79  77 120  69 106  41  62 151
  76  68  93 136  66  40  73  84 101  88  59  52  64  95  78 113  60 131
 102 126 119  86  70]


In [5]:
Initial_Origin_Destination_Count = len(Origin_Destination)

In [6]:
# Create dictionary for block and node pairings (identified from BlockDetermination.ipynb)
block_node_dict = {}

# Iterate over the list and add the block ids and node coordinates to the dictionary
for node_id, block_id, block_coord in Node_Block:
    if int(block_id) not in block_node_dict:
        block_node_dict[int(block_id)] = []
    block_node_dict[int(block_id)].append(node_id)

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_91212/2814293574.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if int(block_id) not in block_node_dict:
/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_91212/2814293574.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  block_node_dict[int(block_id)].append(node_id)


In [7]:
Origin_Destination_Node_Added = Origin_Destination.copy()

In [8]:
# Map nodes to work and home blocks

Origin_Destination_Node_Added['w_node_id'] = Origin_Destination_Node_Added['w_geocode'].map(block_node_dict)
Origin_Destination_Node_Added['h_node_id'] = Origin_Destination_Node_Added['h_geocode'].map(block_node_dict)

In [9]:
# Remove rows where the node columns are missing (i.e. the block is not in the node dictionary)

Origin_Destination_Node_Added = Origin_Destination_Node_Added[Origin_Destination_Node_Added['w_node_id'].notnull()]
Origin_Destination_Node_Added = Origin_Destination_Node_Added[Origin_Destination_Node_Added['h_node_id'].notnull()]

In [10]:
# Explode the node columns so that each row has a single node coordinate

Origin_Destination_Node_Added = Origin_Destination_Node_Added.explode('w_node_id')
Origin_Destination_Node_Added = Origin_Destination_Node_Added.explode('h_node_id')

In [11]:
Unique_Block_Combinations_Count = len(Origin_Destination_Node_Added.index.unique())
Unique_Node_Combinations_Count = len(Origin_Destination_Node_Added)

In [12]:
# Shift node coordinate column locations

Origin_Destination_Node_Added.insert(1, 'w_node_id', Origin_Destination_Node_Added.pop('w_node_id'))
Origin_Destination_Node_Added.insert(3, 'h_node_id', Origin_Destination_Node_Added.pop('h_node_id'))

In [13]:
print(Origin_Destination_Node_Added.head())

              w_geocode w_node_id       h_geocode h_node_id  S000  SA01  SA02  \
1931506  60371011101000     31165  60371011101007     30391     1     0     1   
1931506  60371011101000     31165  60371011101007     30589     1     0     1   
1931506  60371011101000     31166  60371011101007     30391     1     0     1   
1931506  60371011101000     31166  60371011101007     30589     1     0     1   
1931507  60371011101000     31165  60371011222004     31070     1     0     0   

         SA03  SE01  SE02  SE03  SI01  SI02  SI03  createdate  
1931506     0     0     0     1     0     0     1    20230321  
1931506     0     0     0     1     0     0     1    20230321  
1931506     0     0     0     1     0     0     1    20230321  
1931506     0     0     0     1     0     0     1    20230321  
1931507     1     0     1     0     0     0     1    20230321  


In [14]:
# check if there are any rows where w_geocode and h_geocode are the same

same_code = Origin_Destination_Node_Added[Origin_Destination_Node_Added['w_geocode'] == Origin_Destination_Node_Added['h_geocode']]
print(len(same_code))

75791


In [15]:
# check if there are any rows where w_node_id and h_node_id are the same

same_id = Origin_Destination_Node_Added[Origin_Destination_Node_Added['w_node_id'] == Origin_Destination_Node_Added['h_node_id']]
print(len(same_id))

7117


In [16]:
# Remove rows where w_node_id and h_node_id are the same

Origin_Destination_Node_Added = Origin_Destination_Node_Added[Origin_Destination_Node_Added['w_node_id'] != Origin_Destination_Node_Added['h_node_id']]

Unique_Node_Combinations_Count_Final = len(Origin_Destination_Node_Added)

In [17]:
print('LODES dataset origin/destination block combination count: ' + str(Initial_Origin_Destination_Count) + '\nRelevant blocks combination count: ' + str(Unique_Block_Combinations_Count) + '\nRelevant nodes combination count: ' + str(Unique_Node_Combinations_Count) + '\nRelevant nodes combination count (same node to node removed): ' + str(Unique_Node_Combinations_Count_Final))

LODES dataset origin/destination block combination count: 14962890
Relevant blocks combination count: 650320
Relevant nodes combination count: 5380799
Relevant nodes combination count (same node to node removed): 5373682


In [18]:
Origin_Destination_Node_Added

,w_geocode,w_node_id,h_geocode,h_node_id,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
1931506,60371011101000,31165,60371011101007,30391,1,0,1,0,0,0,1,0,0,1,20230321
1931506,60371011101000,31165,60371011101007,30589,1,0,1,0,0,0,1,0,0,1,20230321
1931506,60371011101000,31166,60371011101007,30391,1,0,1,0,0,0,1,0,0,1,20230321
1931506,60371011101000,31166,60371011101007,30589,1,0,1,0,0,0,1,0,0,1,20230321
1931507,60371011101000,31165,60371011222004,31070,1,0,0,1,0,1,0,0,0,1,20230321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5953382,60379800311041,31841,60379800311040,32107,1,0,1,0,0,0,1,1,0,0,20230321
5953382,60379800311041,31841,60379800311040,32126,1,0,1,0,0,0,1,1,0,0,20230321
5953382,60379800311041,31841,60379800311040,32141,1,0,1,0,0,0,1,1,0,0,20230321
5953382,60379800311041,31841,60379800311040,32175,1,0,1,0,0,0,1,1,0,0,20230321


In [19]:
# Count where 'w_geocode' and 'h_geocode' are uniquely paired

Origin_Destination_Node_Added['unique_pair'] = Origin_Destination_Node_Added['w_geocode'].astype(str) + Origin_Destination_Node_Added['h_geocode'].astype(str)

In [20]:
# Count how many times each unique pair appears, and add it as a column

Origin_Destination_Node_Added['unique_pair_count'] = Origin_Destination_Node_Added.groupby('unique_pair')['unique_pair'].transform('count')

In [21]:
# Origin_Destination_Node_Added['S000_adjusted'] = Origin_Destination_Node_Added['S000'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SA01_adjusted'] = Origin_Destination_Node_Added['SA01'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SA02_adjusted'] = Origin_Destination_Node_Added['SA02'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SA03_adjusted'] = Origin_Destination_Node_Added['SA03'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SE01_adjusted'] = Origin_Destination_Node_Added['SE01'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SE02_adjusted'] = Origin_Destination_Node_Added['SE02'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SE03_adjusted'] = Origin_Destination_Node_Added['SE03'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SI01_adjusted'] = Origin_Destination_Node_Added['SI01'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SI02_adjusted'] = Origin_Destination_Node_Added['SI02'] / Origin_Destination_Node_Added['unique_pair_count']
# Origin_Destination_Node_Added['SI03_adjusted'] = Origin_Destination_Node_Added['SI03'] / Origin_Destination_Node_Added['unique_pair_count']

In [22]:
# List of column names to be adjusted
cols_to_adjust = ['S000', 'SA01', 'SA02', 'SA03', 'SE01', 'SE02', 'SE03', 'SI01', 'SI02', 'SI03']

# Iterate over the columns and create new adjusted columns
for col in cols_to_adjust:
    Origin_Destination_Node_Added[f'{col}_adjusted'] = Origin_Destination_Node_Added[col] / Origin_Destination_Node_Added['unique_pair_count']

In [23]:
Origin_Destination_Node_Added.head(20)

,w_geocode,w_node_id,h_geocode,h_node_id,S000,SA01,SA02,SA03,SE01,SE02,...,S000_adjusted,SA01_adjusted,SA02_adjusted,SA03_adjusted,SE01_adjusted,SE02_adjusted,SE03_adjusted,SI01_adjusted,SI02_adjusted,SI03_adjusted
1931506,60371011101000,31165,60371011101007,30391,1,0,1,0,0,0,...,0.250000,0.0,0.25,0.000000,0.00,0.000000,0.25,0.0,0.0,0.250000
1931506,60371011101000,31165,60371011101007,30589,1,0,1,0,0,0,...,0.250000,0.0,0.25,0.000000,0.00,0.000000,0.25,0.0,0.0,0.250000
1931506,60371011101000,31166,60371011101007,30391,1,0,1,0,0,0,...,0.250000,0.0,0.25,0.000000,0.00,0.000000,0.25,0.0,0.0,0.250000
1931506,60371011101000,31166,60371011101007,30589,1,0,1,0,0,0,...,0.250000,0.0,0.25,0.000000,0.00,0.000000,0.25,0.0,0.0,0.250000
1931507,60371011101000,31165,60371011222004,31070,1,0,0,1,0,1,...,0.083333,0.0,0.00,0.083333,0.00,0.083333,0.00,0.0,0.0,0.083333
1931507,60371011101000,31165,60371011222004,31164,1,0,0,1,0,1,...,0.083333,0.0,0.00,0.083333,0.00,0.083333,0.00,0.0,0.0,0.083333
1931507,60371011101000,31165,60371011222004,41110,1,0,0,1,0,1,...,0.083333,0.0,0.00,0.083333,0.00,0.083333,0.00,0.0,0.0,0.083333
1931507,60371011101000,31165,60371011222004,41125,1,0,0,1,0,1,...,0.083333,0.0,0.00,0.083333,0.00,0.083333,0.00,0.0,0.0,0.083333
1931507,60371011101000,31165,60371011222004,41460,1,0,0,1,0,1,...,0.083333,0.0,0.00,0.083333,0.00,0.083333,0.00,0.0,0.0,0.083333
1931507,60371011101000,31165,60371011222004,43506,1,0,0,1,0,1,...,0.083333,0.0,0.00,0.083333,0.00,0.083333,0.00,0.0,0.0,0.083333


In [24]:
# Print column names of the dataframe

print(Origin_Destination_Node_Added.columns)

Index(['w_geocode', 'w_node_id', 'h_geocode', 'h_node_id', 'S000', 'SA01',
       'SA02', 'SA03', 'SE01', 'SE02', 'SE03', 'SI01', 'SI02', 'SI03',
       'createdate', 'unique_pair', 'unique_pair_count', 'S000_adjusted',
       'SA01_adjusted', 'SA02_adjusted', 'SA03_adjusted', 'SE01_adjusted',
       'SE02_adjusted', 'SE03_adjusted', 'SI01_adjusted', 'SI02_adjusted',
       'SI03_adjusted'],
      dtype='object')


In [25]:
# Dump Origin_Destination_Node_Added_Final as a pickle file

Origin_Destination_Node_Added.to_pickle('Origin_Destination_Node_Added.pkl')

In [38]:
# Export updated csv

# Origin_Destination_Node_Added.to_csv('ca_od_main_JT00_2020_Node_Added.csv', index=False)